In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder

In [5]:
tf.__version__

'2.18.0'

In [6]:
from google.colab import files
uploaded = files.upload()

Saving news_train.csv to news_train.csv


In [7]:
train_df = pd.read_csv("news_train.csv")#here we have the dataset we extracted
train_df.head()

,id,title,author,text,label
0,0,Agent every development say quality.,Angie Henderson,Despite measure ago current. Information last ...,0
1,1,Page southern role movie win her need stop.,Shannon Ray,Product significant world. Term herself law st...,0
2,2,Clear here writer policy news range successful.,Crystal Johnson,Environment decision wall then fire pretty. Th...,1
3,3,Race Mr environment political born.,Jesse Mckay,Seven medical blood personal success medical c...,0
4,4,Within mouth call process water.,Joseph Obrien,Sense ready require human public health tonigh...,0


In [8]:
Y = train_df['label']

In [9]:
group1 = pd.DataFrame(train_df.groupby(['author']).size()).reset_index()
group1.head()

,author,0
0,Alfred Williams,1
1,Alyssa Lee,1
2,Amy Ayala,1
3,Amy Kelly,1
4,Angel Jensen,1


In [10]:
group2 = pd.DataFrame(train_df.groupby(['author'])['label'].sum()).reset_index()
group2.head()

,author,label
0,Alfred Williams,1
1,Alyssa Lee,1
2,Amy Ayala,1
3,Amy Kelly,0
4,Angel Jensen,0


In [11]:
group1.columns = ['author','count']
group1.sort_values(by=['count'], ascending=False).head()

,author,count
33,Crystal Johnson,2
1,Alyssa Lee,1
0,Alfred Williams,1
3,Amy Kelly,1
4,Angel Jensen,1


In [12]:
group1[group1['author']=='Starkman']

,author,count


In [13]:
group2.sort_values(by=['label'], ascending=False).head()

,author,label
0,Alfred Williams,1
1,Alyssa Lee,1
2,Amy Ayala,1
5,Angela Johnson,1
12,Brenda Copeland,1


In [14]:
merge_groups = pd.merge(group1,group2, on='author')
merge_groups.head()

,author,count,label
0,Alfred Williams,1,1
1,Alyssa Lee,1,1
2,Amy Ayala,1,1
3,Amy Kelly,1,0
4,Angel Jensen,1,0


In [15]:
merge_groups['prob_fake'] = merge_groups['label']/merge_groups['count']
merge_groups['prob_fake'].describe()

,prob_fake
count,199.000000
mean,0.434673
std,0.495692
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [16]:
merge_groups[merge_groups['author']=='Pam Key']['prob_fake']

,prob_fake


In [17]:
train_df['title_lower'] = train_df["title"].str.lower()
train_df['title_no_punctuation'] = train_df['title_lower'].str.replace('[^\w\s]','')
train_df['title_no_punctuation'] = train_df["title_no_punctuation"].fillna("fillna")

train_df['text_lower'] = train_df["text"].str.lower()
train_df['text_no_punctuation'] = train_df['text_lower'].str.replace('[^\w\s]','')
train_df['text_no_punctuation'] = train_df["text_no_punctuation"].fillna("fillna")

train_df['author_lower'] = train_df["author"].str.lower()
train_df['author_no_spaces'] = train_df['author_lower'].str.replace(' ','_')

In [18]:
train_df.head()

,id,title,author,text,label,title_lower,title_no_punctuation,text_lower,text_no_punctuation,author_lower,author_no_spaces
0,0,Agent every development say quality.,Angie Henderson,Despite measure ago current. Information last ...,0,agent every development say quality.,agent every development say quality.,despite measure ago current. information last ...,despite measure ago current. information last ...,angie henderson,angie_henderson
1,1,Page southern role movie win her need stop.,Shannon Ray,Product significant world. Term herself law st...,0,page southern role movie win her need stop.,page southern role movie win her need stop.,product significant world. term herself law st...,product significant world. term herself law st...,shannon ray,shannon_ray
2,2,Clear here writer policy news range successful.,Crystal Johnson,Environment decision wall then fire pretty. Th...,1,clear here writer policy news range successful.,clear here writer policy news range successful.,environment decision wall then fire pretty. th...,environment decision wall then fire pretty. th...,crystal johnson,crystal_johnson
3,3,Race Mr environment political born.,Jesse Mckay,Seven medical blood personal success medical c...,0,race mr environment political born.,race mr environment political born.,seven medical blood personal success medical c...,seven medical blood personal success medical c...,jesse mckay,jesse_mckay
4,4,Within mouth call process water.,Joseph Obrien,Sense ready require human public health tonigh...,0,within mouth call process water.,within mouth call process water.,sense ready require human public health tonigh...,sense ready require human public health tonigh...,joseph obrien,joseph_obrien


In [19]:
train_df['author_no_spaces'].head()

,author_no_spaces
0,angie_henderson
1,shannon_ray
2,crystal_johnson
3,jesse_mckay
4,joseph_obrien


In [20]:
max_features=5000 #we set maximum number of words to 5000
maxlen=400 #we set maximum sequence length to 400

In [21]:
tok = tf.keras.preprocessing.text.Tokenizer(num_words=max_features) #again tokenizer step
tok.fit_on_texts(list(train_df['text_no_punctuation'])+list(train_df['title_no_punctuation'])+list(train_df['author_no_spaces'].astype(str))) #fit to cleaned text

In [22]:
print(len(tok.word_index))
vocab_size = len(tok.word_index) + 1

1259


In [23]:
text_df = tok.texts_to_sequences(list(train_df['text_no_punctuation'])) #this is how we create sequences
text_df = tf.keras.preprocessing.sequence.pad_sequences(text_df, maxlen=maxlen) #let's execute pad step

title_df = tok.texts_to_sequences(list(train_df['title_no_punctuation'])) #this is how we create sequences
title_df = tf.keras.preprocessing.sequence.pad_sequences(title_df, maxlen=maxlen)

author_df = tok.texts_to_sequences(list(train_df['author_no_spaces'].astype(str))) #this is how we create sequences
author_df = tf.keras.preprocessing.sequence.pad_sequences(author_df, maxlen=maxlen)

In [24]:
train_df = author_df + title_df + text_df

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df, Y, test_size=0.1, random_state=42)

In [26]:
embedding_dim = 50

In [27]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',#no more categorical_crossentropy
              metrics=['accuracy'])

In [29]:
model.fit(np.array(X_train), np.array(y_train), epochs=1)

6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 110ms/step - accuracy: 0.5257 - loss: 0.6915
